## 1) Decortiquer un code de fine-tuning on a Q&A dataset

On considère ici le dataset Stanford Question Answering Dataset (SQuAD) et va entrainer **distilbert-base-uncased** sur ce dataset. Décortiquons un code typique pour faire cela, nous en profiterons pour voir les aspects de deep learning dont nous allons avoir besoin.

SQuAD est un dataset pour lesquel chaque élément contient trois éléments: le contexte, la question et la réponse. Et dans la version 1 de SQuAD, systématiquement la réponse est contenue dans le contexte. Dans la version 2, ce n'est pas toujours le cas et ca veut dire que le modele doit etre capable de predire qu'il n'y a pas la bonne reponse.
- SQuAD 1.0 dataset https://web.stanford.edu/class/archive/cs/cs224n/cs224n.1174/reports/2749099.pdf
- SQuAD 2.0 dataset https://web.stanford.edu/class/archive/cs/cs224n/cs224n.1194/reports/default/15839661.pdf

Le concept pour fine-tuned in model sur ce jeu de données Q&A c'est de prendre un contexte + question et tenter de prédire la réponse.
**le modèle est donc entraîné à prédire le couple (start_point, end_point)** a partir de contexte + question.

La loss est donc assez simple : c'est **L((start_point_pred, end_point_pred), (start_point, end_point))**.


In [1]:
from datasets import load_dataset
from transformers import DistilBertTokenizerFast, DistilBertForQuestionAnswering, TrainingArguments, Trainer

# Comme d'habitude, nous telechargeons le dataset, tokenizer et le model
dataset = load_dataset("squad")
tokenizer = DistilBertTokenizerFast.from_pretrained("distilbert-base-uncased")
model = DistilBertForQuestionAnswering.from_pretrained("distilbert-base-uncased")

Some weights of DistilBertForQuestionAnswering were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Commençons par regarder comment le dataset est structuré pour mieux comprendre la **loss** (fonction de perte) avec laquelle nous allons entraîner le modele.

In [2]:
print(f"Le context est : {dataset['train']['context'][0]}" + '\n')
print(f"La question est : {dataset['train']['question'][0]}" + '\n')
print(f"La reponse est : {dataset['train']['answers'][0]}" + '\n')

Le context est : Architecturally, the school has a Catholic character. Atop the Main Building's gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.

La question est : To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?

La reponse est : {'text': ['Saint Bernadette Soubirous'], 'answer_start': [515]}



Pour pouvoir commencer a entrainer un modele, comme les models travaillent directement sur les tokens, il faut tokeniser l'input (context + question) et la reponse. Prenons l'exemple d'une question en particulier : 

In [3]:
context = dataset['train']['context'][0]
question = dataset['train']['question'][0]
reponse = dataset['train']['answers'][0]

In [4]:
# Existe-t-il une solution pour tronquer autour de la reponse, connaissant la position de la reponse ?
input = tokenizer(
    question,
    context,
    max_length=384,
    truncation="only_second",
    return_offsets_mapping=True,
    padding="max_length",
)
# 1) max_length=384 : la longueur combinee de question + context est tronquee si depasse cette limite.
# 2) truncation="only_second" : si question + context est tronquee, on tronque le context.
# 3) padding="max_length": On pads la sequence tokenise a la longueur maximale. C'est important pour
# s'assurer que tous les inputs ont la meme longueur (batch processing efficace lors de l'entrainement du modele)
# 4) return_offsets_mapping=True : on explique juste apres a quoi ils correspondent et pourquoi c'est interessant.

La sortie du tokenizer **input** est un dictionnaire avec trois clefs 'input_ids', 'attention_mask', 'offset_mapping':
- **'input_ids'** : corresponds aux indices des tokens de context + question. input['input_ids'] est une liste de longueur 384 et qui finit par des zeros (qui correspondent au token [PAD]).
- **'offset_mapping'** : corresponds aux indices des tokens dans input['input_ids'] avec (idx_start, idx_end) dans la string initiale context + question.
- **'attention_mask'** : Une liste de 0 ou de 1. C'est pour dire quels sont les tokens utiles pour l'attention. Globalement c'est juste pour indiquer au modele qu'il n'y a pas besoin de processer les tokens de [PAD]...


Exemple de Text: "My name is Wolfgang"
Tokens (fake) : ["My", "name", "is", "Wolfgang"]

Le offset mapping pour cet exemple serait
offset_mapping = [(0, 2), (3, 7), (8, 10), (11, 19), (0, 0), ..., (0, 0)]

Parce que

- Le token "My" commence au caractere 0 et termine au caractere 2.
- Le token "name" commence au caractere 3 et termine au caractere 7.
- Le token "is" commence au caractere 9 et termine au caractere 10.
- Le token "Wolfgang" commence au caractere 11 et termine au caractere 19.


In [5]:
print(list(input.keys()))

['input_ids', 'attention_mask', 'offset_mapping']


In [6]:
print(len(input['input_ids']))
print(len(input['attention_mask']))
print(input['attention_mask'])

384
384
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

Pour l'instant nous avons juste tokenisé l'input, il nous reste à tokeniser la réponse pour pouvoir calculer l'erreur du modèle. Idealement, notre objectif est de rajouter deux champs a l'input l'information (start_idx_answer_in_context, end_idx_answer_in_context.

Faisons cela sur un exemple en particulier.

La méthode **sequence_ids()** de l'objet input renvoit une liste indiquant si chaque token decrit dans input est part 
- de la question (valeur 0)
- du context (valeur 1)
- est un special tokens (valeur None)


In [8]:
# Le premier None correspong a BOS tandis que le second corresponds a la transition de la question vers le contexte.
# Les derniers corresponds au [PAD] tokens.
sequence_ids = input.sequence_ids()
print(sequence_ids)

[None, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, None, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, 

A partir de cette liste on peut trouver l'index du premier token associé au contexte et trouver l'indice du dernier token associé au contexte.

In [9]:
# Find the start index of the context
context_start = sequence_ids.index(1)

# Find the end index of the context
context_end = len(sequence_ids) - 1 - sequence_ids[::-1].index(1)
print(f'The context starts at token {context_start} and ends at token {context_end}')

The context starts at token 17 and ends at token 174


Il est intéressant de reconstruire à partir du champs input_ids de input la liste des tokens. Ca permet de comprendre aussi les differents special tokens qui sont utilises. On voit
- **[CLS]** : nous n'en parlons pas pour l'instant.
- **[SEP]** : c'est le token de separation entre la question et le contexte.
- **[PAD]** : c'est le token de padding.

Noter que dans le processus de tokenization process utilisé par BERT ou DistilBERT le symbole **##** est utilisé pour indiquer le cas lorsqu'un token est sous-mot ou la continuation d'un mot précédent.

In [10]:
# On peut a partir de input_ids reconstruire la context + question ...

# On commence par passer des indices des tokens a une liste de token
tokens = tokenizer.convert_ids_to_tokens(input['input_ids'])
print(tokens)

['[CLS]', 'to', 'whom', 'did', 'the', 'virgin', 'mary', 'allegedly', 'appear', 'in', '1858', 'in', 'lou', '##rdes', 'france', '?', '[SEP]', 'architectural', '##ly', ',', 'the', 'school', 'has', 'a', 'catholic', 'character', '.', 'atop', 'the', 'main', 'building', "'", 's', 'gold', 'dome', 'is', 'a', 'golden', 'statue', 'of', 'the', 'virgin', 'mary', '.', 'immediately', 'in', 'front', 'of', 'the', 'main', 'building', 'and', 'facing', 'it', ',', 'is', 'a', 'copper', 'statue', 'of', 'christ', 'with', 'arms', 'up', '##rai', '##sed', 'with', 'the', 'legend', '"', 've', '##ni', '##te', 'ad', 'me', 'om', '##nes', '"', '.', 'next', 'to', 'the', 'main', 'building', 'is', 'the', 'basilica', 'of', 'the', 'sacred', 'heart', '.', 'immediately', 'behind', 'the', 'basilica', 'is', 'the', 'gr', '##otto', ',', 'a', 'marian', 'place', 'of', 'prayer', 'and', 'reflection', '.', 'it', 'is', 'a', 'replica', 'of', 'the', 'gr', '##otto', 'at', 'lou', '##rdes', ',', 'france', 'where', 'the', 'virgin', 'mary', 

In [11]:
# Filter out special tokens and padding
filtered_tokens = [token for token in tokens if token not in tokenizer.all_special_tokens]

# Manually join the tokens into a string
prompt = ' '.join(filtered_tokens).replace(' ##', '')

print("Reconstructed prompt without special tokens and padding:", prompt)

Reconstructed prompt without special tokens and padding: to whom did the virgin mary allegedly appear in 1858 in lourdes france ? architecturally , the school has a catholic character . atop the main building ' s gold dome is a golden statue of the virgin mary . immediately in front of the main building and facing it , is a copper statue of christ with arms upraised with the legend " venite ad me omnes " . next to the main building is the basilica of the sacred heart . immediately behind the basilica is the grotto , a marian place of prayer and reflection . it is a replica of the grotto at lourdes , france where the virgin mary reputedly appeared to saint bernadette soubirous in 1858 . at the end of the main drive ( and in a direct line that connects through 3 statues and the gold dome ) , is a simple , modern stone statue of mary .


Maintenant que nous avons compris un petit mieux les différents champs de input et la structure de context/question/answer, revenons à notre objectif principal : **processer l'input pour donner le couple (start, end) de la réponse dans le contexte et dans le cas ou ce n'est pas possible comprendre ce qu'il faut donner en output**.

In [12]:
context = dataset['train']['context'][0]
question = dataset['train']['question'][0]
answer = dataset['train']['answers'][0]

input = tokenizer(
        question,
        context,
        max_length=384,
        truncation="only_second",
        return_offsets_mapping=True,
        padding="max_length",
    )

In [13]:
# On peut deja commencer par trouver la position (en terme de nombre de caracteres 
# de la reponse dans le context.
start_pos = context.find(answer['text'][0])
end_pos = start_pos + len(answer['text'][0])

In [14]:
context[start_pos:end_pos]

'Saint Bernadette Soubirous'

In [15]:
# En fait, le dictionnaire answer contient aussi le champs answer.
start_char = answer["answer_start"][0]
end_char = start_char + len(answer["text"][0])
# Attention: on raisonne sur les string et pas sur les tokens.

In [16]:
context[start_char:end_char]

'Saint Bernadette Soubirous'

Il nous faut maintenant gerer la situation ou la partie du contexte qui contient l'information a ete tronque

In [17]:
offset_mapping = input["offset_mapping"]

In [18]:
print(offset_mapping)

[(0, 0), (0, 2), (3, 7), (8, 11), (12, 15), (16, 22), (23, 27), (28, 37), (38, 44), (45, 47), (48, 52), (53, 55), (56, 59), (59, 63), (64, 70), (70, 71), (0, 0), (0, 13), (13, 15), (15, 16), (17, 20), (21, 27), (28, 31), (32, 33), (34, 42), (43, 52), (52, 53), (54, 58), (59, 62), (63, 67), (68, 76), (76, 77), (77, 78), (79, 83), (84, 88), (89, 91), (92, 93), (94, 100), (101, 107), (108, 110), (111, 114), (115, 121), (122, 126), (126, 127), (128, 139), (140, 142), (143, 148), (149, 151), (152, 155), (156, 160), (161, 169), (170, 173), (174, 180), (181, 183), (183, 184), (185, 187), (188, 189), (190, 196), (197, 203), (204, 206), (207, 213), (214, 218), (219, 223), (224, 226), (226, 229), (229, 232), (233, 237), (238, 241), (242, 248), (249, 250), (250, 252), (252, 254), (254, 256), (257, 259), (260, 262), (263, 265), (265, 268), (268, 269), (269, 270), (271, 275), (276, 278), (279, 282), (283, 287), (288, 296), (297, 299), (300, 303), (304, 312), (313, 315), (316, 319), (320, 326), (327

In [19]:
print(sequence_ids)

[None, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, None, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, 

In [20]:
# sequence_ids = inputs.sequence_ids(i)
# context_start = sequence_ids.index(1)
# context_end = len(sequence_ids) - sequence_ids[::-1].index(1) - 1

Plusieurs situations peuvent subvenir maintenat :
- **Si le contexte a été tronqué par le tronque par le tokenizer, la partie du contexte contenant la réponse n'existe plus et nous sommes dans le cas ou la réponse à la question n'est pas contenue dans le contexte ! Dans ce cas, nous disons que (start, end) = (0,0), ce qui revient à assigner la réponse au token CLS, on comprend enfin l'utilité du token CLS !**
- La condition `offset[context_start][0] > end_char or offset[context_end][1] < start_char` vérifie exactement si nous sommes dans la situation décrite précédemment.
- Ensuite il faut réfléchir un petit peu pour déduire à partir des positions des caractères (start, end) de la réponse, la position des indices de token dans le prompt.

In [37]:
if offset_mapping[context_start][0] > end_char or offset_mapping[context_end][1] < start_char:
    start_position = 0
    end_position = 0
else:
    start_position = next(i for i, x in enumerate(offset_mapping) if x[0] <= start_char < x[1])
    end_position = next(i for i, x in enumerate(offset_mapping) if x[0] < end_char <= x[1])

Maintenant nous pouvons écrire la fonction qui va faire ce processing au niveau du dataset.

In [22]:
import datasets
import tqdm
from transformers import DistilBertTokenizerFast

# Load the tokenizer
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

# Function to preprocess the data
def preprocess_function(examples: datasets.DatasetDict):
    '''
    '''

    questions = [q.strip() for q in examples['question']]
    contexts = [c.strip() for c in examples['context']]

    inputs = tokenizer(
        questions,
        contexts,
        max_length=384,
        truncation="only_second",
        return_offsets_mapping=True,
        padding="max_length",
    )

    offset_mapping = inputs.pop("offset_mapping")
    answers = examples["answers"]
    start_positions = []
    end_positions = []
    
    for i, offset in enumerate(offset_mapping):
        answer = answers[i]
        start_char = answer["answer_start"][0]
        end_char = start_char + len(answer["text"][0])

        sequence_ids = inputs.sequence_ids(i)
        context_start = sequence_ids.index(1)
        context_end = len(sequence_ids) - sequence_ids[::-1].index(1) - 1
    
        if offset[context_start][0] > start_char or offset[context_end][1] < end_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            start_pos = [i for i, x in enumerate(offset) if x[0] <= start_char <= x[1]][0]
            end_pos = [i for i, x in enumerate(offset) if x[0] <= end_char <= x[1]][0]
            start_positions.append(start_pos)
            end_positions.append(end_pos)
    
    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions

    return inputs

In [23]:
# Apply the preprocessing function to the dataset
encoded_dataset = dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/10570 [00:00<?, ? examples/s]

Le processus d'entrainement lui est tres peu detaille :
Le trainer de huggingFace : https://huggingface.co/docs/transformers/en/main_classes/trainer

In [39]:
!pip install tensorboardX tensorboard > /dev/null 2>&1

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [70]:
# %load_ext tensorboard
%reload_ext tensorboard
%tensorboard --logdir ./logs

Reusing TensorBoard on port 6006 (pid 11867), started 0:00:30 ago. (Use '!kill 11867' to kill it.)

In [40]:
!pip install accelerate -U > /dev/null 2>&1

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [26]:
from datasets import load_dataset
from transformers import DistilBertTokenizerFast, DistilBertForQuestionAnswering, Trainer, TrainingArguments
from tensorboardX import SummaryWriter  # or use `torch.utils.tensorboard.SummaryWriter` if using PyTorch

# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',          # folder pour stoker les checkpoints
    evaluation_strategy="epoch",     # evaluer apres chaque epoque
    learning_rate=2e-5,              # learning rate
    per_device_train_batch_size=16,  # batch size pour entrainement
    per_device_eval_batch_size=16,   # batch size pour evaluation
    num_train_epochs=3,              # nombre d'epoques
    weight_decay=0.01,               # regularization L2 ...
    logging_dir='./logs',            # pour stocker les logs auxquels nous allons acceder via tensorboard
    logging_steps=1,
    report_to="tensorboard",
)

# Initialize the Trainer
trainer = Trainer(
    model=model,                         # Le model HF Transformers a entrainer
    args=training_args,                  # les arguments d'entrainement
    train_dataset=encoded_dataset['train'],   # le dataset d'entrainement
    eval_dataset=encoded_dataset['validation'],   # le dataset d'evaluation
)

# Start training
trainer.train()

# Evaluate the model
results = trainer.evaluate()
print(results)

/opt/mamba/lib/python3.12/site-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 

## 2) Parameter Efficient Fine-tuning on a Q&A dataset

Quelques idees derriere QLoRA ou LoRA:
- On **freeze** un grand nombre de poids du modeles. Cela evite notamment le *catastrophic forgetting*.
- LoRA remplace certaines matrices de poids par des matrices de low-rank dans les layers des transformers. La matrice initial de poids n'est pas modifiee mais plutot les deux matrices qui composent chacun des poids.
- LoRA ne fait cela que pour certains types de matrices de poids (typiquement les matrices associes au calcul de score d'attention).
- En plus de tout cela, QLora vient quantizer les poids a INT4.


Quelques tres bonnes ressources pour comprendre tous les details: 
- https://medium.com/@dillipprasad60/qlora-explained-a-deep-dive-into-parametric-efficient-fine-tuning-in-large-language-models-llms-c1a4794b1766
- https://towardsdatascience.com/qlora-how-to-fine-tune-an-llm-on-a-single-gpu-4e44d6b5be32

In [38]:
!pip install transformers datasets accelerate peft  > /dev/null 2>&1

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [28]:
from transformers import DistilBertForQuestionAnswering, TrainingArguments, Trainer
from peft import (
    LoraConfig,
    get_peft_model,
    prepare_model_for_kbit_training)

Les differents packages pour facilement faire du fine-tuning de LLM
- **datasets** = package de HuggingFace pour avoir acces a de nombreux datasets.
- **peft** = Parameter Efficient Fine-Tuning. Developpe par HuggingFace.
- **AutoModelForCausalLM** = C'est une facon de retrouver l'architecture du pretrained model a partir du nom du modele. Developpe par HuggingFace.
- **transformers** = un des packages les plus connu de HuggingFace qui regroupe de nombreuses fonctionnalites autour des transformers.

In [42]:
# Load the pre-trained model
model = DistilBertForQuestionAnswering.from_pretrained("distilbert-base-uncased")
model.to('cuda')

Some weights of DistilBertForQuestionAnswering were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


DistilBertForQuestionAnswering(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
      

We have to prepare the models for the pretraining in the following ways :
- **model.train()** : preparer le modele pour l'entrainement, et c'est surtout a faire lorsque le modele a des layers de dropout.
- **model.gradient_checkpointing_enable()** : permet de faire du **gradient checkpointing**. Il s'agit d'une technique dans les reseaux de neurones pour reduire l'usage de memoire pendant l'entrainement.
- **prepare_model_for_kbit_training** : préparer des modèles pour un entraînement avec une précision réduite, en particulier avec des largeurs de bits inférieures (par exemple, un entraînement 8 bits ou 4 bits).

**Dropout** = drop out pendant l'entrainement un certain nombre de neurones des reseaux de neurones.

In [43]:
model.train() # model in training mode (dropout modules are activated)

DistilBertForQuestionAnswering(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
      

In [44]:
# enable gradient check pointing
model.gradient_checkpointing_enable()

# enable quantized training
model = prepare_model_for_kbit_training(model)

**Il nous faut maintenant configurer le type d'entrainement LoRA que nous avons envie de pratiquer.**

We have to find the **target modules** that are the best to use in order to perform the LoRA technique. En effet, il faut preciser la configuration de **LoraConfig** et notamment la variable **target_modules**.

- **lora_alpha** = c'est un **scaling factor** pour injecter les matrices low-rank dans l'entrainement.
- **target_modules** = Les parties du reseau de neurone ou on va modifier les matrices de poids.

In [46]:
# CE CODE DONNE UNE ERREUR PARCE QUE J"AI MIS N'IMPORTE QUOI POUR LA VARIABLE TARGET_MODULES
# Define LoRA configuration
lora_config = LoraConfig(
    r=8,  # Rank of the adaptation matrices
    lora_alpha=32,  # Scaling factor
    target_modules=["attention.q_lin", "attention.k_lin", "attention.v_lin"],  # Modules to apply LoRA to
    lora_dropout=0.1,
)

# On applique LoRA au model = on ajoute nos matrices low-rank que l'on va apprendre
model = get_peft_model(model, lora_config)

Pour avoir le nom de toutes les layers voici comment on procede 

In [47]:
# Print all submodule names
for name, module in model.named_modules():
    print(name)


base_model
base_model.model
base_model.model.distilbert
base_model.model.distilbert.embeddings
base_model.model.distilbert.embeddings.word_embeddings
base_model.model.distilbert.embeddings.position_embeddings
base_model.model.distilbert.embeddings.LayerNorm
base_model.model.distilbert.embeddings.dropout
base_model.model.distilbert.transformer
base_model.model.distilbert.transformer.layer
base_model.model.distilbert.transformer.layer.0
base_model.model.distilbert.transformer.layer.0.attention
base_model.model.distilbert.transformer.layer.0.attention.dropout
base_model.model.distilbert.transformer.layer.0.attention.q_lin
base_model.model.distilbert.transformer.layer.0.attention.q_lin.base_layer
base_model.model.distilbert.transformer.layer.0.attention.q_lin.lora_dropout
base_model.model.distilbert.transformer.layer.0.attention.q_lin.lora_dropout.default
base_model.model.distilbert.transformer.layer.0.attention.q_lin.lora_A
base_model.model.distilbert.transformer.layer.0.attention.q_lin.

Les matrices QKV sont les éléments auxquels nous avons envie d'appliquer les modification LoRA parce que ce sont des matrices a peu pres quadratiques en la taille de la contexte window.

In [48]:
# Define LoRA configuration
lora_config = LoraConfig(
    r=8,  # Rank of the adaptation matrices
    lora_alpha=32,  # Scaling factor
    target_modules=["attention.q_lin", "attention.k_lin", "attention.v_lin"],  # Modules to apply LoRA to
    lora_dropout=0.1,
)

# Apply LoRA to the model
model = get_peft_model(model, lora_config)

In [49]:
# trainable parameter count
model.print_trainable_parameters()

trainable params: 221,184 || all params: 66,585,602 || trainable%: 0.3322


Maintenant nous pouvons entrainer comme precedement le model.

In [71]:
from datasets import load_dataset
from transformers import DistilBertTokenizerFast, DistilBertForQuestionAnswering, Trainer, TrainingArguments
from tensorboardX import SummaryWriter  # or use `torch.utils.tensorboard.SummaryWriter` if using PyTorch

# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',          # output directory
    evaluation_strategy="epoch",     # evaluate each epoch
    learning_rate=2e-5,              # learning rate
    per_device_train_batch_size=16,  # batch size for training
    per_device_eval_batch_size=16,   # batch size for evaluation
    num_train_epochs=3,              # number of training epochs
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=1,
    report_to="tensorboard",
    remove_unused_columns=False,
)

# Initialize the Trainer
trainer = Trainer(
    model=model,                                 # the instantiated 🤗 Transformers model to be trained
    args=training_args,                          # training arguments, defined above
    train_dataset=encoded_dataset['train'],      # training dataset
    eval_dataset=encoded_dataset['validation'],  # evaluation dataset
)

# Start training
trainer.train()

# Evaluate the model
results = trainer.evaluate()
print(results)

/opt/mamba/lib/python3.12/site-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


RuntimeError: Could not infer dtype of dict

In [73]:
# ['id', 'title', 'context', 'question', 'answers', 'input_ids', 'attention_mask', 'start_positions', 'end_positions']
encoded_dataset['train'][0]['start_positions']

130

## 4) Travailler sur des taches autres que du Q&A.

Nous pouvons aussi fine-tuner des modeles sur d'autres datasets 
- NLI (Natural Language Inference)
- STS (Semantic Textual Similarity)

**Exercice**: Faire du fine-tuning sur le dataset **snli**.

In [ ]:
from datasets import load_dataset

dataset = load_dataset('snli')

In [ ]:
dataset

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

model_name = 'roberta-base'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=3)  # 3 labels pour NLI: entailment

Ici la preprocess_function est beaucoup plus simple (NLI n'est qu'une tache de classification a trois classes).

In [ ]:
def preprocess_function(examples):
    return tokenizer(examples['premise'], examples['hypothesis'], truncation=True, padding='max_length')

encoded_dataset = dataset.map(preprocess_function, batched=True)

In [ ]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=encoded_dataset['train'],
    eval_dataset=encoded_dataset['validation'],
    tokenizer=tokenizer
)

In [ ]:
trainer.train()

In [ ]:
model.save_pretrained('./fine-tuned-model')
tokenizer.save_pretrained('./fine-tuned-model')

## 4) Fine-tuning sur d'autres Q&A dataset

Il y a  de nombreux autres jeux de donnees de Q&A ou le type de taches est semblable a celui pour SQuAD: predire le couple (start_pos, end_pos) . Par exemple :
- https://huggingface.co/datasets/google-research-datasets/natural_questions?row=0 NQ : Natural Questions.
- https://huggingface.co/datasets/mandarjoshi/trivia_qa?row=8 TriviaQ

Mais une difficulte est que chacun de ces datasets a des structures differentes. **Exercice**: pour deux autres datasets **trivia-qa-triplet** et **Quora**, les explorer et comprendre quelle pourrait la tache d'entrainement.
- https://huggingface.co/datasets/toughdata/quora-question-answer-dataset Quora.
https://huggingface.co/datasets/sentence-transformers/trivia-qa-triplet

Commencons par **trivia-qa-triplet**.

In [ ]:
import pdb

import tqdm
from datasets import load_dataset
from transformers import DistilBertTokenizerFast, DistilBertForQuestionAnswering


dataset = load_dataset("trivia_qa", "rc")

In [ ]:
dataset = load_dataset("trivia_qa", "rc")

In [ ]:
dataset['train']